In [ ]:
# --- Importación de librerías principales ---

# Manejo de datos tabulares y operaciones numéricas
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Librería para generar números aleatorios (no esencial, pero útil para reproducibilidad o simulaciones)
import random

# --- Instalación y carga de librería financiera ---
# numpy-financial se usa para cálculos financieros como TIR, VAN, etc.
!pip install numpy-financial
import numpy_financial as npf

# --- Estadística ---
# pearsonr permite calcular la correlación de Pearson (relación lineal entre dos variables)
from scipy.stats import pearsonr
import scipy.stats as stats

# --- Deep Learning (TensorFlow / Keras) ---
import tensorflow as tf
from tensorflow.keras.models import Sequential   # Modelo secuencial (apilamiento de capas)
from tensorflow.keras.layers import Dense         # Capa totalmente conectada (fully connected)
from tensorflow.keras.optimizers import SGD       # Optimizador: descenso del gradiente estocástico

# --- Métricas ---
# confusion_matrix: matriz de confusión para evaluar la clasificación binaria
from sklearn.metrics import confusion_matrix

In [ ]:
# --- Carga y exploración inicial del conjunto de datos ---

# Carga el archivo CSV en un DataFrame de pandas.
# 'dataset.csv' debe encontrarse en el mismo directorio del notebook.
df = pd.read_csv("dataset.csv")

# Muestra el contenido completo del DataFrame (primeras filas por defecto en notebooks).
df

# Genera un resumen estadístico de las variables numéricas del dataset:
# incluye media, desviación estándar, valores mínimo, máximo y percentiles (25%, 50%, 75%).
df.describe()


In [ ]:

# Se extraen las columnas relevantes del DataFrame original y se convierten en arreglos de NumPy.
# Cada arreglo representa una variable clave del proceso de desarrollo de software.
datosDevs = df['Devs'].values
datosSprint = df['Sprint'].values
datosDiseno = df['Diseno'].values
datosDesarrollo = df['Desarrollo'].values
datosControl = df['Control'].values

# Se realiza una copia profunda del DataFrame original para no modificar los datos base.
df_copy = df.copy(deep=True)


# --- Paso 3: Definición de parámetros para Bootstrapping ---

n = len(datosDevs)           # Tamaño de la muestra original
n_bootstrap = 2000           # Número de iteraciones Bootstrap (muestras con reemplazo)

# Listas para almacenar los resultados generados durante las simulaciones
medias_bootstrapDevs = []
medias_bootstrapSprint = []
medias_bootstrapDiseno = []
medias_bootstrapDesarrollo = []
medias_bootstrapControl = []


# En cada iteración se seleccionan valores aleatorios (con reemplazo) de cada variable
# y se calcula una nueva duración estimada (Duracion) combinando las fases del proyecto.
for _ in range(n_bootstrap):

    # Muestreo con reemplazo de cada variable
    muestra_Devs = random.choice(datosDevs)
    muestra_Sprint = random.choice(datosSprint)
    muestra_Diseno = random.choice(datosDiseno)
    muestra_Desarrollo = random.choice(datosDesarrollo)
    muestra_Control = random.choice(datosControl)

    # Cálculo de una nueva "Duración" simulada a partir de las variables seleccionadas
    muestra_Duracion = (muestra_Diseno + muestra_Desarrollo + muestra_Control) * muestra_Sprint

    # Se crea un nuevo registro (fila) con los valores simulados
    new_row = pd.DataFrame({
        'Devs': [muestra_Devs],
        'Sprint': [muestra_Sprint],
        'Diseno': [muestra_Diseno],
        'Desarrollo': [muestra_Desarrollo],
        'Control': [muestra_Control],
        'Duracion': [muestra_Duracion]
    })

    # Se concatena la nueva fila al DataFrame copia
    df_copy = pd.concat([df_copy, new_row], ignore_index=True)



# Se calculan los percentiles 2.5 y 97.5 de las duraciones simuladas,
# lo que representa un intervalo de confianza del 95% para la media de la variable Duracion.
confianza_95 = np.percentile(df_copy['Duracion'].values, [2.5, 97.5])



# Se construye un histograma para observar la distribución de las duraciones simuladas.
plt.hist(df_copy['Duracion'].values, bins=50, edgecolor='black', alpha=0.7)

# Se agregan líneas verticales que marcan los límites del intervalo de confianza.
plt.axvline(confianza_95[0], color='red', linestyle='dashed', linewidth=2, label=f'2.5% Percentil: {confianza_95[0]:.2f}')
plt.axvline(confianza_95[1], color='red', linestyle='dashed', linewidth=2, label=f'97.5% Percentil: {confianza_95[1]:.2f}')

plt.title('Distribución de medias de las muestras Bootstrap')
plt.xlabel('Media de Duración')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()


# Se imprime el intervalo de confianza obtenido, el cual cuantifica la incertidumbre
# asociada a la media estimada de la duración total de los proyectos simulados.
print(f"Intervalo de confianza del 95% para la media de Duracion: ({confianza_95[0]:.2f}, {confianza_95[1]:.2f})")

In [ ]:
# --- Simulación de flujos y cálculo de TIR por fila (proyecto) ---

final_df = pd.DataFrame()

# Recorremos df_copy para construir los flujos semanales y calcular la TIR por proyecto simulado
for index, row in df_copy.iterrows():
    # Límite superior de filas procesadas (control de tiempo/recursos)
    if index < 2013:
        # --- Extracción de parámetros por fila ---
        devs = row['Devs']           # número de desarrolladores
        duracion = int(row['Duracion'])  # duración en sprints/semanas (asegurar entero)
        
        # --- Parámetros financieros (convención: negativos = egresos, positivos = ingresos) ---
        costo_inicial = -53343.47    # inversión inicial 
        costo_fijo = -7089.26        # costo fijo por periodo 
        
        # --- Flujo de caja semanal ---
        # Se construye un vector: [flujo_0, flujo_1, ..., flujo_duracion]
        flujo = []
        flujo.append(costo_inicial)  # flujo en t=0
        
        # Nota: (devs*40*-125) = costo de horas hombre; (devs*40*175) = ingreso por horas facturadas
        # En conjunto equivale a devs*40*(175-125)= devs*40*50, pero lo mantenemos explícito por claridad contable.
        for i in range(duracion):
            flujo_semanal = (devs * 40 * -125) + (devs * 40 * 175) + costo_fijo
            flujo.append(flujo_semanal)
        
        # --- Cálculo de la TIR (npf.irr usa convención de signos del flujo) ---
        tir = npf.irr(flujo)  # devuelve tasa en formato decimal (p.ej. 0.23 = 23%)
        
        # --- Construcción de la nueva fila con resultados ---
        # IMPORTANTE: guardamos TIR numérica (float) para análisis estadístico posterior.
        row['TIR'] = tir
        row['Aceptado'] = "SI" if (tir * 100) > 20 else "NO"  # regla de decisión (hurdle rate 20%)
        
        # Se agrega la fila al DataFrame final
        final_df = pd.concat([final_df, pd.DataFrame(row).T], ignore_index=True)

# Vista rápida
final_df

# --- Persistencia a CSV y recarga (opcional para trazabilidad) ---
final_df.to_csv('dataset_final.csv', index=False)
final_df = pd.read_csv("dataset_final.csv")

# Resumen estadístico
final_df
final_df.describe()

# --- Preparación para análisis de correlación ---
# Removemos columnas no numéricas antes de correlación
# (Si 'Aceptado' existe, la eliminamos; si 'TIR' se quiere excluir, se puede eliminar aquí)
cols_a_eliminar = [c for c in ['TIR', 'Aceptado', 'TIR %'] if c in final_df.columns]
final_df.drop(columns=cols_a_eliminar, inplace=True)

# Matriz de correlación (solo numéricos)
correlation_matrix = final_df.corr(numeric_only=True)

# --- Función para matriz de p-values (correlaciones de Pearson) ---
def calculate_pvalues(df_numeric):
    # Asegurar solo columnas numéricas y sin NaN para evitar errores en pearsonr
    df_numeric = df_numeric.select_dtypes(include=[np.number]).dropna()
    p_values = pd.DataFrame(np.ones((df_numeric.shape[1], df_numeric.shape[1])),
                            columns=df_numeric.columns, index=df_numeric.columns)

    for i in range(len(df_numeric.columns)):
        for j in range(i, len(df_numeric.columns)):
            corr, p_value = pearsonr(df_numeric.iloc[:, i], df_numeric.iloc[:, j])
            p_values.iloc[i, j] = p_value
            p_values.iloc[j, i] = p_value

    return p_values

# Cálculo de p-values
p_values_matrix = calculate_pvalues(final_df)

# --- Visualizaciones ---

# Mapa de calor de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Mapa de Calor de Correlación')
plt.show()

# Mapa de calor de p-values
plt.figure(figsize=(10, 8))
sns.heatmap(p_values_matrix, annot=True, cmap='Blues', vmin=0, vmax=1)
plt.title('Mapa de Calor de P-values')
plt.show()


In [ ]:
# Se obtiene el intervalo de confianza del 95% para la variable 'Devs' (número de desarrolladores),
# utilizando los percentiles 2.5 y 97.5 de la distribución empírica.
confianza_95 = np.percentile(final_df['Devs'].values, [2.5, 97.5])

# Se calcula la media de la distribución de desarrolladores.
media = np.mean(final_df['Devs'].values)


# ---  Visualización de la distribución de desarrolladores ---

plt.hist(final_df['Devs'].values, bins=10, edgecolor='black', alpha=0.7)

# Líneas verticales para marcar los percentiles (intervalo de confianza del 95%)
plt.axvline(confianza_95[0], color='red', linestyle='dashed', linewidth=2, 
            label=f'2.5% Percentil: {confianza_95[0]:.2f}')
plt.axvline(confianza_95[1], color='red', linestyle='dashed', linewidth=2, 
            label=f'97.5% Percentil: {confianza_95[1]:.2f}')

# Línea verde sólida para la media
plt.axvline(media, color='green', linestyle='solid', linewidth=3, 
            label=f'Media: {media:.2f}')

# Títulos y etiquetas del gráfico
plt.xlabel('Número de Desarrolladores (Devs)')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()


# ---  Definición de la distribución triangular por columna numérica ---

def triangle_distribution(df):
    """
    Calcula los parámetros de la distribución triangular (mínimo, moda, máximo)
    para cada columna numérica de un DataFrame.

    Args:
        df : DataFrame de entrada.

    Returns:
        DataFrame con los parámetros (min, mode, max) de cada variable numérica.
    """

    # Selecciona solo las columnas numéricas del DataFrame
    numeric_cols = df.select_dtypes(include=np.number).columns
    triangle_params = {}

   # Itera por cada columna numérica y calcula los tres parámetros
    for col in numeric_cols:
        min_val = df[col].min()             # Valor mínimo
        max_val = df[col].max()             # Valor máximo
        mode_val = df[col].mode()[0]        # Moda (toma el primer valor si hay varias)
        triangle_params[col] = {
            'min': min_val,
            'mode': mode_val,
            'max': max_val
        }

    # Retorna los resultados en un DataFrame transpuesto (una fila por variable)
    return pd.DataFrame(triangle_params).T


triangle_df = triangle_distribution(final_df)
triangle_df


# Se genera una nueva columna binaria llamada 'Aceptado':
# True si la TIR es mayor o igual al 20% (proyecto aceptado), False en caso contrario.
final_df['Aceptado'] = final_df['TIR'] >= 20


# Se agrupan los datos por 'Devs' (número de desarrolladores) y el estado de aceptación.
resultado = final_df.groupby(['Devs', 'Aceptado'])['Devs'].count().unstack(fill_value=0)

# Renombrar las columnas para mayor claridad en la tabla resultante
resultado.columns = ['No Aceptados', 'Aceptados']

# Mostrar la tabla resumen (frecuencia de aceptación por tamaño de equipo)
print(resultado)

In [ ]:
# Se carga nuevamente el dataset final con los resultados del cálculo de TIR
final_df = pd.read_csv("dataset_final.csv")

# Se crea una copia de trabajo del DataFrame para manipularlo sin alterar el original
model_df = final_df.copy()

# --- Generación de la variable objetivo (target) ---

# Se crea una nueva columna llamada 'Aceptar' que funcionará como variable de salida (Y) para la red neuronal.
# La lógica aplicada es:
#   Si la TIR > 20 → se etiqueta con 1 (proyecto aceptado)
#   Si la TIR ≤ 20 → se etiqueta con 0 (proyecto no aceptado)
# Este umbral del 20% actúa como la "tasa mínima aceptable de retorno" o *hurdle rate*.
model_df["Aceptar"] = [1 if x > 20 else 0 for x in model_df['TIR']]

# --- Limpieza de variables no necesarias para el entrenamiento ---

# Se eliminan las columnas que no deben incluirse como entradas del modelo:
#  - 'TIR' y 'TIR %' → variables dependientes del resultado financiero
#  - 'Aceptado' → etiqueta redundante con 'Aceptar'
#  - 'Duracion' → puede eliminarse si no se utilizará como predictor
model_df.drop(columns=['TIR', 'Aceptado', 'TIR %', 'Duracion'], inplace=True, errors='ignore')

# --- Visualización del DataFrame final listo para el modelo ---
# En este punto, model_df contiene exclusivamente variables independientes (X)
# y la variable de salida binaria 'Aceptar' para el entrenamiento supervisado.
model_df


In [ ]:
# --- Datos de ejemplo (X: características, y: etiquetas) ---

# X: variables predictoras; y: etiqueta binaria (0/1) que indica si el proyecto se acepta
X = model_df.drop(columns=['Aceptar'])   # Entrada (características)
y = model_df['Aceptar']                  # Salida (etiquetas)

# --- División 70/30 (opcional) ---
# NOTA: el cliente pidió entrenar con 100% de los casos; por eso se deja comentado.
#Inicialmente se utilizo este entrenamiento, despues del analisis de los resultados con el  cliente este preifirio utilizar el 100% de los datos
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# --- Definición del modelo (arquitectura 5–5–1) ---
model = Sequential()

# Capa oculta con 5 neuronas y 5 entradas (ReLU)
model.add(Dense(5, input_dim=5, activation='relu'))

# Capa de salida (1 neurona con Sigmoid para probabilidad de clase positiva)
model.add(Dense(1, activation='sigmoid'))

# Compilación del modelo: pérdida binaria, métrica de accuracy y optimizador SGD
model.compile(optimizer=SGD(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# --- Entrenamiento ---
# Entrena con el 100% de los datos (decisión del cliente). verbose=False para no saturar la salida.
historial = model.fit(X, y, epochs=100, batch_size=1, verbose=False)

# --- Evaluación (sobre el mismo set de entrenamiento) ---
# Nota: medir en el mismo set tiende a sobreestimar el desempeño (optimismo).
pérdida, precisión = model.evaluate(X, y, verbose=False)
print(f"Pérdida: {pérdida:.4f}")
print(f"Precisión: {precisión:.4f}")

# --- Predicciones y umbral de decisión ---
# Se obtienen probabilidades en [0,1] y se umbralizan a clase {0,1}.
predicciones = model.predict(X, verbose=False)

print("\nProbabilidades / puntuaciones del modelo:")
print(predicciones.flatten())

# Umbral de decisión elevado para priorizar alta certeza en positivos (minimizar falsos positivos)
umbral = 0.90
predicciones_binarias = (predicciones > umbral).astype(int)

# --- Comparación con etiquetas reales ---
print("\nComparación con las etiquetas reales:")
print(f"Predicciones binarias (umbral={umbral}): {predicciones_binarias.flatten()}")
print(f"Etiquetas reales: {y.values}")

# --- Matriz de confusión ---
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, predicciones_binarias)

print("\nMatriz de confusión:")
print(cm)

# --- Curvas de entrenamiento (loss y accuracy) ---
plt.plot(historial.history['loss'])
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Pérdida durante el entrenamiento')
plt.show()

plt.plot(historial.history['accuracy'])
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Precisión durante el entrenamiento')
plt.show()

# --- Heatmap de la matriz de confusión ---
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', cbar=False,
            xticklabels=['Not Accepted', 'Accepted'],
            yticklabels=['Not Accepted', 'Accepted'])
plt.xlabel('Predictions')
plt.ylabel('Actual Results')
plt.title('Matriz de Confusión')

# Leyendas auxiliares (ubicación relativa al heatmap)
plt.text(0.5, 1.05, 'FN: False Negatives', ha='center', va='center', fontsize=10, color='black', transform=plt.gca().transAxes)
plt.text(0.88, 1.05, 'TP: True Positives', ha='center', va='center', fontsize=10, color='black', transform=plt.gca().transAxes)
plt.text(0.5, -0.10, 'TN: True Negatives', ha='center', va='center', fontsize=10, color='black', transform=plt.gca().transAxes)
plt.text(0.88, -0.10, 'FP: False Positives', ha='center', va='center', fontsize=10, color='black', transform=plt.gca().transAxes)

plt.tight_layout()
plt.show()
